In [1]:
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from io import BytesIO
from zipfile import ZipFile
import urllib.request
import os

In [4]:
browser = webdriver.Chrome('/usr/local/bin/chromedriver')

In [5]:
browser.get('https://s3.amazonaws.com/tripdata/index.html')

In [6]:
#get all links
all_links = [link.get_attribute('href') for link in browser.find_elements_by_tag_name('a')]

In [7]:
# extract necessary links
linksJC_17 = [link for link in all_links if 'JC' in link if '2017' in link]
linksJC_18 = [link for link in all_links if 'JC' in link if '2018' in link]

In [8]:
# function for downloading, unziping and dataframing data from a link
def create_df(link):
    
    url = urllib.request.urlopen(link)
    output = open('temporary.zip', 'wb')    
    output.write(url.read())
    output.close()
    dataframe = pd.read_csv('temporary.zip')
    
    if (len(dataframe.columns) == 15):
        dataframe.columns = ['Trip Duration (sec)', 'Start Time', 'Stop Time', 'Start Station ID',
       'Start Station Name', 'Start Station Latitude',
       'Start Station Longitude', 'End Station ID', 'End Station Name',
       'End Station Latitude', 'End Station Longitude', 'Bike ID', 'User Type',
       'Birth_Year', 'Gender']
    else:
        dataframe.columns = ['Trip Duration (sec)', 'Start Time', 'Stop Time', 'Start Station ID',
       'Start Station Name', 'Start Station Latitude',
       'Start Station Longitude', 'End Station ID', 'End Station Name',
       'End Station Latitude', 'End Station Longitude', 'Bike ID',
       'Localized Value', 'User Type', 'Birth_Year', 'Gender']
        dataframe = dataframe.drop('Localized Value',1)
        
    print(link)
    os.remove('temporary.zip')
    
    return dataframe

In [9]:
# function for cleaning and preparing df
def prepare_df(df):
    
#     drop n/a
    df = df.dropna(how='any').reset_index(drop=True)
    
#     change data types
    df['Birth_Year'] = df.Birth_Year.astype(int)
    df['Start Time'] = pd.to_datetime(df['Start Time'])
    df['Stop Time'] = pd.to_datetime(df['Stop Time'])
    
#     add Dge column
    df['Age'] = 2018 - df['Birth_Year']
    
#     exclude ages > 90 years
    df = df[df['Age'] < 90]
    
    return df

In [10]:
# create empty df
jc17 = pd.DataFrame()
jc18 = pd.DataFrame()

In [11]:
# append to new df
for link in linksJC_17:
    temporary_df = create_df(link)
    jc17 = jc17.append(temporary_df, ignore_index=True, sort=False)

for link in linksJC_18:
    temporary_df = create_df(link)
    jc18 = jc18.append(temporary_df, ignore_index=True, sort=False)

https://s3.amazonaws.com/tripdata/JC-201701-citibike-tripdata.csv.zip
https://s3.amazonaws.com/tripdata/JC-201702-citibike-tripdata.csv.zip
https://s3.amazonaws.com/tripdata/JC-201703-citibike-tripdata.csv.zip
https://s3.amazonaws.com/tripdata/JC-201704-citibike-tripdata.csv.zip
https://s3.amazonaws.com/tripdata/JC-201705-citibike-tripdata.csv.zip
https://s3.amazonaws.com/tripdata/JC-201706-citibike-tripdata.csv.zip
https://s3.amazonaws.com/tripdata/JC-201707-citibike-tripdata.csv.zip
https://s3.amazonaws.com/tripdata/JC-201708%20citibike-tripdata.csv.zip
https://s3.amazonaws.com/tripdata/JC-201709-citibike-tripdata.csv.zip
https://s3.amazonaws.com/tripdata/JC-201710-citibike-tripdata.csv.zip
https://s3.amazonaws.com/tripdata/JC-201711-citibike-tripdata.csv.zip
https://s3.amazonaws.com/tripdata/JC-201712-citibike-tripdata.csv.zip
https://s3.amazonaws.com/tripdata/JC-201801-citibike-tripdata.csv.zip
https://s3.amazonaws.com/tripdata/JC-201802-citibike-tripdata.csv.zip
https://s3.amazona

In [12]:
# clean and prepare df
jc17 = prepare_df(jc17)
jc18 = prepare_df(jc18)

In [13]:
jc17.head()

,Trip Duration (sec),Start Time,Stop Time,Start Station ID,Start Station Name,Start Station Latitude,Start Station Longitude,End Station ID,End Station Name,End Station Latitude,End Station Longitude,Bike ID,User Type,Birth_Year,Gender,Age
0,148,2017-01-01 00:21:32,2017-01-01 00:24:01,3276,Marin Light Rail,40.714584,-74.042817,3185,City Hall,40.717733,-74.043845,24575,Subscriber,1983,1,35
1,1283,2017-01-01 00:24:35,2017-01-01 00:45:58,3183,Exchange Place,40.716247,-74.033459,3198,Heights Elevator,40.748716,-74.040443,24723,Subscriber,1978,1,40
2,372,2017-01-01 00:38:19,2017-01-01 00:44:31,3183,Exchange Place,40.716247,-74.033459,3211,Newark Ave,40.721525,-74.046305,24620,Subscriber,1989,1,29
3,1513,2017-01-01 00:38:37,2017-01-01 01:03:50,3194,McGinley Square,40.725340,-74.067622,3271,Danforth Light Rail,40.692640,-74.088012,24668,Subscriber,1961,1,57
4,639,2017-01-01 01:47:52,2017-01-01 01:58:31,3183,Exchange Place,40.716247,-74.033459,3203,Hamilton Park,40.727596,-74.044247,26167,Subscriber,1993,1,25


In [14]:
jc18.head()

,Trip Duration (sec),Start Time,Stop Time,Start Station ID,Start Station Name,Start Station Latitude,Start Station Longitude,End Station ID,End Station Name,End Station Latitude,End Station Longitude,Bike ID,User Type,Birth_Year,Gender,Age
0,132,2018-01-01 00:01:46,2018-01-01 00:03:58,3186,Grove St PATH,40.719586,-74.043117,3211,Newark Ave,40.721525,-74.046305,29590,Subscriber,1964,1,54
1,560,2018-01-01 01:27:17,2018-01-01 01:36:38,3276,Marin Light Rail,40.714584,-74.042817,3269,Brunswick & 6th,40.726012,-74.050389,29613,Subscriber,1989,2,29
2,294,2018-01-01 01:29:03,2018-01-01 01:33:58,3195,Sip Ave,40.730743,-74.063784,3225,Baldwin at Montgomery,40.723659,-74.064194,31940,Subscriber,1994,1,24
3,197,2018-01-01 01:59:32,2018-01-01 02:02:49,3211,Newark Ave,40.721525,-74.046305,3278,Monmouth and 6th,40.725685,-74.048790,31949,Subscriber,1964,1,54
4,932,2018-01-01 02:06:18,2018-01-01 02:21:50,3183,Exchange Place,40.716247,-74.033459,3199,Newport Pkwy,40.728745,-74.032108,31929,Subscriber,1992,1,26


In [15]:
jc17.dtypes

Trip Duration (sec)                 int64
Start Time                 datetime64[ns]
Stop Time                  datetime64[ns]
Start Station ID                    int64
Start Station Name                 object
Start Station Latitude            float64
Start Station Longitude           float64
End Station ID                      int64
End Station Name                   object
End Station Latitude              float64
End Station Longitude             float64
Bike ID                             int64
User Type                          object
Birth_Year                          int64
Gender                              int64
Age                                 int64
dtype: object

In [16]:
jc18.dtypes

Trip Duration (sec)                 int64
Start Time                 datetime64[ns]
Stop Time                  datetime64[ns]
Start Station ID                    int64
Start Station Name                 object
Start Station Latitude            float64
Start Station Longitude           float64
End Station ID                      int64
End Station Name                   object
End Station Latitude              float64
End Station Longitude             float64
Bike ID                             int64
User Type                          object
Birth_Year                          int64
Gender                              int64
Age                                 int64
dtype: object

In [17]:
jcdf = jc17.append(jc18, ignore_index=True, sort=False)

In [18]:
jcdf.count()

Trip Duration (sec)        413781
Start Time                 413781
Stop Time                  413781
Start Station ID           413781
Start Station Name         413781
Start Station Latitude     413781
Start Station Longitude    413781
End Station ID             413781
End Station Name           413781
End Station Latitude       413781
End Station Longitude      413781
Bike ID                    413781
User Type                  413781
Birth_Year                 413781
Gender                     413781
Age                        413781
dtype: int64

In [19]:
jcdf.to_csv('data/jcbike_from_url.csv', encoding='utf-8', index=False)